# Importation des librairies

In [3]:
! pip install -q pandas-Profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 617.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 592.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 1.0 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 844.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 1.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 1.3 MB/s eta 0:00:0000:0

In [54]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
from io import StringIO
from sklearn.impute import KNNImputer

# Importation des bases

## Import de la base de données de consommation d'électricité  et des données météorologiques via API de l'OpenDataSoft

L'éxécution de la requête API prend assez de temps

In [55]:
def get_data_ODS(dataset_name):
    """
    Fonction qui permet de récupérer des données via l'API d'OpenDataSoft
    Elle retourne un dataframe
    
    """
    df = pd.DataFrame()
    # URL de base pour accéder à l'API d'ODS
    base_url_ODS = "https://odre.opendatasoft.com/api/explore/v2.1"
    
    dataset_path = f"/catalog/datasets/{dataset_name}/exports/json?lang=fr&timezone=Europe%2FBerlin"
    url = f"{base_url_ODS}{dataset_path}"
    
    # Exécution de la requête GET
    
    response = requests.get(url)
    # Vérification si la requête a réussi
    if response.status_code == 200:
        # Extraction des données
        data = response.json()
        # Conversion des résultats en DataFrame
        df = pd.DataFrame(data)
    else:
        print(f"Erreur lors de la requête: {response.status_code}")

    return df


def get_data_ODSW(dataset_name, output = "json"):
    """
    Fonction qui permet de récupérer des données via l'API  publique d'OpenDataSoft
    Elle retourne un dataframe
    
    """
    df = pd.DataFrame()
    # URL de base pour accéder à l'API d'ODS
    base_url_ODS = "https://public.opendatasoft.com/api/explore/v2.1"
    
    dataset_path = f"/catalog/datasets/{dataset_name}/exports/{output}?lang=fr&timezone=Europe%2FBerlin"
    url = f"{base_url_ODS}{dataset_path}"
    
    # Exécution de la requête GET
    response = requests.get(url)
    # Vérification si la requête a réussi
    if response.status_code == 200:
        # Extraction des données
        if output == "json":
            data = response.json()
            # Conversion des résultats en DataFrame
            df = pd.DataFrame(data)
        else:
            data = response.json()
            geometries = [shape(feature['geometry']) for feature in data['features']]
            properties = [feature['properties'] for feature in data['features']]
    
            # Créer un GeoDataFrame en combinant les géométries et les propriétés
            df = gpd.GeoDataFrame(geometry=geometries, data=properties)
    else:
        print(f"Erreur lors de la requête: {response.status_code}")

    return df

In [18]:
consumption_dataset_name = "consommation-quotidienne-brute-regionale"
weather_dataset_name = "donnees-synop-essentielles-omm"
geographic_dataset_name = "georef-france-commune"
data_consumption = get_data_ODS(consumption_dataset_name)
data_weather = get_data_ODSW(weather_dataset_name)
data_geo = get_data_ODSW(geographic_dataset_name, output="geojson")

Erreur lors de la requête: 404


,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,altitude,libgeo,codegeo,nom_epci,code_epci,nom_dept,code_dep,nom_reg,code_reg,mois_de_l_annee
0,07558,2010-01-05T10:00:00+01:00,100280.0,-50.0,5,260.0,1.5,275.75,275.75,100.0,...,712,Millau,12145,CC de Millau Grands Causses,241200567,Aveyron,12,Occitanie,76,1
1,61976,2010-01-05T10:00:00+01:00,100990.0,NaN,None,NaN,NaN,305.45,299.05,69.0,...,7,None,None,None,None,None,None,None,None,1
2,07027,2010-01-05T13:00:00+01:00,100720.0,-190.0,8,200.0,3.6,273.65,271.75,87.0,...,67,Carpiquet,14137,CU Caen la Mer,200065597,Calvados,14,Normandie,28,1
3,07110,2010-01-05T13:00:00+01:00,100750.0,-230.0,8,210.0,4.1,276.95,272.55,73.0,...,94,Guipavas,29075,Brest Métropole,242900314,Finistère,29,Bretagne,53,1
4,07591,2010-01-05T13:00:00+01:00,NaN,NaN,None,NaN,NaN,274.45,269.05,67.0,...,871,Embrun,05046,CC Serre-Ponçon,200067742,Hautes-Alpes,05,Provence-Alpes-Côte d'Azur,93,1


## Import d'une base de données complémentaire (celle des régions, communes, et départements de France

Cette importation va se faire à travers une requête via l'API de Datagouv

In [24]:
#Identifiant du dataset
dataset_id = "4a7c837bb6da8e363604082bcc8b2e504cf08038"
dataset_sha1 = "dbe8a621-a9c4-4bc3-9cae-be1699c5ff25"

# URL de base pour accéder à l'API
base_url = "https://www.data.gouv.fr/api/1/"

# Chemin pour accéder aux enregistrements du dataset
dataset_path = f"datasets/r/{dataset_sha1}?dataset={dataset_id}"

# Construction de l'URL complète
url = f"{base_url}{dataset_path}"

# Exécution de la requête GET
response = requests.get(url)

# Vérification si la requête a réussi
if response.status_code == 200:
    # Lecture du contenu CSV
    data_communes = pd.read_csv(StringIO(response.content.decode('utf-8')))
else:
    print(f"Erreur lors de la requête: {response.status_code}")

On enregistre la base pour ne plus avoir à éxécuter la requête API à chaque étape

In [25]:
data_consumption.to_csv('data_consumption.csv', index=False)
data_communes.to_csv('data_communes.csv', index=False)
data_weather.to_csv('data_weather.csv', index = False)

In [35]:
data_consumption = pd.read_csv('data_consumption.csv')
data_communes = pd.read_csv('data_communes.csv')
data_weather = pd.read_csv('data_weather.csv')

Les dataframes de consommation et météorologiques  étant sous forme de séries temporelles, on place les dates en index.

In [36]:
data_consumption['date'] = pd.to_datetime(data_consumption['date'])
data_consumption.set_index('date', inplace=True)

In [37]:
data_communes.isna().sum()

code_commune_INSEE          0
nom_commune_postal          0
code_postal                 0
libelle_acheminement        0
ligne_5                 35944
latitude                  269
longitude                 269
code_commune                7
article                 36621
nom_commune                 0
nom_commune_complet         0
code_departement            7
nom_departement           267
code_region               267
nom_region                267
dtype: int64

In [38]:
col_del = pd.Index(['date_heure', 'consommation_brute_gaz_grtgaz', 'statut_grtgaz', 'consommation_brute_gaz_terega', 'statut_terega','consommation_brute_gaz_totale','statut_rte', 'consommation_brute_totale'])
data_consumption.drop(col_del, axis=1,inplace=True)
data_consumption.isna().sum()

heure                                 0
code_insee_region                     0
region                                0
consommation_brute_electricite_rte    0
dtype: int64

## Aggrégation des consommations par date et par région

In [39]:
data_consumption1 = data_consumption.groupby(['code_insee_region','date'], as_index = False)
data_consumption2 = data_consumption.groupby(['code_insee_region','date','heure'], as_index = False)

In [40]:
data_consumption1 = data_consumption1['consommation_brute_electricite_rte'].sum()
data_consumption2 = data_consumption2['consommation_brute_electricite_rte'].sum()

/tmp/ipykernel_498/3628389353.py:1: FutureWarning: A grouping was used that is not in the columns of the DataFrame and so was excluded from the result. This grouping will be included in a future version of pandas. Add the grouping as a column of the DataFrame to silence this warning.
  data_consumption1 = data_consumption1['consommation_brute_electricite_rte'].sum()
/tmp/ipykernel_498/3628389353.py:2: FutureWarning: A grouping was used that is not in the columns of the DataFrame and so was excluded from the result. This grouping will be included in a future version of pandas. Add the grouping as a column of the DataFrame to silence this warning.
  data_consumption2 = data_consumption2['consommation_brute_electricite_rte'].sum()


In [44]:
data_consumption1.head()

,code_insee_region,consommation_brute_electricite_rte
0,11,399392
1,11,492157
2,11,487111
3,11,470053
4,11,433732


### Traitement de la base weather

In [45]:
nouveaux_noms_colonnes={'numer_sta':'num_station', 'date':'date_UTC', 'pmer':'pression_mer', 'tend':'variation_pression_3h', 'cod_tend':'type_tendance_baro', 'dd':'direction_vent', 'ff':'vitesse_vent', 't':'temperature', 'td':'point_de_rosee',
       'u':'humidite', 'vv':'visibilite_horizontale', 'ww':'temps_present', 'w1':'temps_passe_1', 'w2':'temps_passe_2', 'n':'nebulosite_totale', 'nbas':'nebulosite_nuage_etage_inf', 'hbas':'hauteur_base_nuage_etage_inf', 'cl':'type_nuage_etage_inf', 'cm':'type_nuage_etage_moyen', 'ch':'type_nuage_etage_sup',
       'pres':'pression_station', 'niv_bar':'niveau_barometriq', 'geop':'geopotentiel', 'tend24':'variation_pression_24h', 'tn12':'temperature_min_sur_12h', 'tn24':'temperature_min_sur_24h', 'tx12':'temperature_max_sur_12h', 'tx24':'temperature_max_sur_24h',
       'tminsol':'temperature_min_sol_sur_12h', 'sw':'methode_mesure_temperature_thermometre_mouille', 'tw':'temperature_thermometre_mouille', 'raf10':'rafales_10_dernieres_minutes', 'rafper':'rafales_sur_une_periode', 'per':'periode_mesure_rafales', 'etat_sol':'etat_du_sol', 'ht_neige':'hauteur_totale_couche_neige/glace_au_sol',
       'ssfrai':'hauteur_neige_fraiche', 'perssfrai':'periode_mesure_neige_fraiche', 'rr1':'precipitation_dans_1_derniere_heure', 'rr3':'precipitation_dans_3_derniere_heure', 'rr6':'precipitation_dans_6_derniere_heure', 'rr12':'precipitation_dans_12_derniere_heure', 'rr24':'precipitation_dans_24_derniere_heure', 'phenspe1':'phenomene_special_1',
       'phenspe2':'phenomene_special_2', 'phenspe3':'phenomene_special_3', 'phenspe4':'phenomene_special_4', 'nnuage1':'nebulosite_couche_nuageuse_1', 'ctype1':'type_de_nuage_1', 'hnuage1':'hauteur_de_base_nuage_1',
       'nnuage2':'nebulosite_couche_nuageuse_2', 'ctype2':'type_de_nuage_2', 'hnuage2':'hauteur_de_base_nuage_2', 'nnuage3':'nebulosite_couche_nuageuse_3', 'ctype3':'type_de_nuage_3', 'hnuage3':'hauteur_de_base_nuage_3',
       'nnuage4':'nebulosite_couche_nuageuse_4', 'ctype4':'type_de_nuage_4', 'hnuage4':'hauteur_de_base_nuage_4', 'coordonnees':'coordonnees', 'nom':'nom_commune',
       'type_de_tendance_barometrique':'type_de_tendance_barometrique', 'temps_passe_1':'temps_passe_1', 'temps_present':'temps_present', 'tc':'temperature_degre_celcius',
       'tn12c':'temperature_celcius_min_sur_12h', 'tn24c':'temperature_celcius_min_sur_24h', 'tx12c':'temperature_celcius_max_sur_12h', 'tx24c':'temperature_celcius_max_sur_24h', 'tminsolc':'temperature_celcius_min_sol_sur_12h', 'latitude':'latitude', 'longitude':'longitude',
       'altitude':'altitude', 'libgeo':'libelle_geolocalisation', 'codegeo':'code_geolocalisation', 'nom_epci':'nom_EPCI', 'code_epci':'code_EPCI', 'nom_dept':'nom_departement',
       'code_dep':'code_departement', 'nom_reg':'nom_region', 'code_reg':'code_region', 'mois_de_l_annee':'mois'}

In [46]:
base_meteo_modifie_1 = data_weather.rename(columns=nouveaux_noms_colonnes)

## Identification des variables météoroliques à retenir

Il est question d'identifier les les variables météorologiques se reférant aux espects retenus Après une revue de la littérature:
* __Température :__ les variables identifiées sont la température en degré celcius ('temperature_degre_celcius'), la température minimale sur les 24 dernières heures ('temperature_celcius_min_sur_24h'), et la température maximale sur les 24 dernières heures ('temperature_celcius_max_sur_24h')
* __Vitesse du vent :__ la variable identifiée est la vitesse du vent ('vitesse_vent')
* __Couverture nuageuse :__ la variable identifié est la nébulosité totale et renvoie à la quantité totale de nuages couvrant le ciel ou plus simplement le degré de couverture nuageuse dans le ciel ('nebulosite_totale')
* __Humidité :__ la variable identifiée est l'indice d'humidité ('humidite')
* __Rayonnement global :__ la variable identifiée pour capter cet aspect est la distance maximale à laquelle des objets peuvent être clairement discernés à l'horizon ('visibilite_horizontale')

### Vérification et gestion des valeurs manquantes pour les variables d'agrégation

#### Vérification des régions et départements manquants

In [47]:
print(base_meteo_modifie_1['nom_region'].isnull().sum())
print(base_meteo_modifie_1['code_region'].isnull().sum())
print(base_meteo_modifie_1['nom_departement'].isnull().sum())
print(base_meteo_modifie_1['code_departement'].isnull().sum())
print(base_meteo_modifie_1['nom_commune'].isnull().sum())

242582
242582
242582
242582
0


On constate que tous les noms de commune sont renseignés, mais certains noms de région et département ne le sont pas. Nous allons donc utiliser les noms de commune pour renseigner les départements et régions manquants.

Nous explorons la possibilité de completer les régions et départements manquants à partir de la base de données des communes, régions et départements.

In [48]:
test=pd.merge(base_meteo_modifie_1, data_communes, on=['nom_commune'], how='left')
print(test['nom_region_y'].isnull().sum())
print(test['code_region_y'].isnull().sum())
print(test['nom_departement_y'].isnull().sum())
print(test['code_departement_y'].isnull().sum())
print(test['nom_commune'].isnull().sum())

2352712
2352712
2352712
2352712
0


## Agrégation des variables météorologiques

In [49]:
base_meteo_modifie_2=base_meteo_modifie_1

Il est ici question de construire les variables qui seront utilisées dans les analyses. Les données seront agrégées au niveau régional et à une fréquence journalière.

Il est ici question de construire les variables qui seront utilisées dans les analyses. Les données seront agrégées au niveau régional et à une fréquence journalière.

In [50]:
def extraire_sous_chaine(chaine, debut, fin):
    """
    Extrait la sous-chaîne de 'debut' à 'fin' (inclus) de la chaîne donnée.
    """
    if debut < 0 or fin >= len(chaine):
        raise ValueError("Indices de début ou de fin invalides.")
    return chaine[debut : fin + 1]

Nous appliquons la fonction à la base:

In [51]:
base_meteo_modifie_2['jour']=base_meteo_modifie_2['date_UTC'].apply(extraire_sous_chaine,debut=0,fin=9)
base_meteo_modifie_2['heure']=base_meteo_modifie_2['date_UTC'].apply(extraire_sous_chaine,debut=11,fin=18)

## Agrégation des variables à l'échelle départementale

Tout d'abord, agrégeons les variables 'temperature_degre_celcius','vitesse_vent', 'nebulosite_totale', 'humidite', 'visibilite_horizontale'

In [52]:
agregation_dep_mean = base_meteo_modifie_2.groupby(['jour','code_departement']).agg({'temperature_degre_celcius': 'mean', 'vitesse_vent': 'mean','nebulosite_totale':'mean','visibilite_horizontale':'mean'})
agregation_dep_mean = agregation_dep_mean.rename(columns={'jour':'jour','code_departement':'code_departement','temperature_degre_celcius':'temperature_degre_celcius_departement','vitesse_vent':'vitesse_vent_departement','nebulosite_totale':'nebulosite_totale_departement','visibilite_horizontale':'visibilite_horizontale_departement'})

In [53]:
base_meteo_modifie_3 = pd.merge(base_meteo_modifie_2, agregation_dep_mean, on=['jour','code_departement'], how='left')

Imputation des variables manquantes : KNN Imputer

On se dit que les zones ayant des données semblables météorologiques "se ressemblent"

On vérifie d'abord que toutes les variables sont numériques; 

In [34]:
# Cas des données de consommation
data_consumption.dtypes

# Cas des données des communes
data_communes.dtypes


code_commune_INSEE       object
nom_commune_postal       object
code_postal               int64
libelle_acheminement     object
ligne_5                  object
latitude                float64
longitude               float64
code_commune            float64
article                  object
nom_commune              object
nom_commune_complet      object
code_departement         object
nom_departement          object
code_region             float64
nom_region               object
dtype: object

On convertit les colonnes

In [ ]:
cols = []
for col in cols:
    df[col] = df[col].astype(float)


On applique ensuite le KNN imputer

In [32]:

# Initialiser l'imputeur KNN
cons_imputer = KNNImputer(n_neighbors=2)
com_imputer = KNNImputer(n_neighbors=2)

# Appliquer l'imputation
data_consumption_imputed = pd.DataFrame(cons_imputer.fit_transform(data_consumption), columns=data_consumption.columns)
data_communes_imputed = pd.DataFrame(com_imputer.fit_transform(data_communes), columns = data_communes.columns)


ValueError: could not convert string to float: '2019-02-01T06:00:00+01:00'

### Traitement de la base geographique pour le calcul des surfaces

In [ ]:
data_geo.columns

In [ ]:
#On recupère les colonnes importantes de la base de données
cols = ['geo_point_2d', 'reg_code','reg_name', 'dep_code', 'dep_name', 'com_code', 'com_name', 'geometry']
data_geo2 = data_geo[cols]
cols_to_mod = ['reg_code','reg_name', 'dep_code', 'dep_name', 'com_code', 'com_name']
#On effectue un nettoyage des données contenues dans les colonnes
data_geo2[cols_to_mod] = data_geo2[cols_to_mod].applymap(lambda x: ''.join(filter(lambda char: char.isalnum() or char in ['_', '-', '@', '#'], str(x))))

### Calcul de la superficie

In [ ]:
# On divise par par 1e6 pour obtenir la superficie par commune en km²
data_geo2['superficie_comm'] = data_geo2['geometry'].area / 1e6  

#On fait un regroupement par département, et on calcule la superficie par département en sommant les autres superficies
data_geo2['superficie_dep'] = data_geo2.groupby('dep_code')['superficie_comm'].transform('sum')